In [1]:
import os
import re
from os.path import join
from io import StringIO
from glob import glob
import shutil

import sympy
import pandas as pd
from sympy import Symbol, sqrt, cos, pi, symbols
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from collections import OrderedDict

import qnet
from qnet.algebra import *

import QDYN

from src.notebook_plots_v1 import plot_bs_decay, display_hamiltonian, display_eq, show_summary_dicke
from src.single_sided_network_v1 import network_slh
from src.dicke_single_model_v2 import write_dicke_single_model, err_dicke_single
from src.doit_actions_v1 import run_traj_prop

from doit.tools import register_doit_as_IPython_magic

In [2]:
qnet.init_printing()

In [3]:
DOIT_CONFIG = {
    'backend': 'json',
    'dep_file': '.doit_db/2017-06-26_mg_dicke1_compare_evaluations.json',
}
register_doit_as_IPython_magic()

$
\newcommand{ket}[1]{\vert #1 \rangle}
\newcommand{bra}[1]{\langle #1 \vert}
\newcommand{Op}[1]{\hat{#1}}
$

# How should the error be evaluated?

## Set up runfolders

In [4]:
SOURCE_RF = './data/dicke1_QSL_1traj_xxx/rf_2nodes_T1_50'

In [5]:
DATA_ROOT = './data/dicke1_compare_evaluations'

In [6]:
params_data_str = r'''
# n_nodes     T  n_trajs      mcwf  seed
        2  1.50        1     False     0
        2  1.50      100      True     0
        2  1.50      100      True     1
        2  1.50      100      True     2
        2  1.50      700      True     3
        2  1.50     1000      True     4
        2  1.50     2000      True     5
        2  1.50     4000      True     6
        2  1.50     8000      True     7
        2  1.50    16000      True     8
        2  1.50    32000      True     9
        2  1.50    64000      True    10
        2  1.50   128000      True    11
        2  1.50   128000      True    12
'''

def read_bool(s):
    """convert string to boolean"""
    true_vals = ['1', 'true']
    false_vals = ['0', 'false']
    s = s.lower()
    if s in true_vals:
        return True
    elif s in false_vals:
        return False
    else:
        raise ValueError("Cannot convert '%s'" % s)

params_df = pd.read_fwf(
        StringIO(params_data_str), comment='#', header=1,
        names=['n_nodes', 'T', 'n_trajs', 'mcwf', 'seed'],
        converters={
            'n_nodes': int, 'T': float, 'n_trajs': int,
            'mcwf': read_bool, 'seed': int})


In [7]:
params_df

,n_nodes,T,n_trajs,mcwf,seed
0,2,1.5,1,False,0
1,2,1.5,100,True,0
2,2,1.5,100,True,1
3,2,1.5,100,True,2
4,2,1.5,700,True,3
5,2,1.5,1000,True,4
6,2,1.5,2000,True,5
7,2,1.5,4000,True,6
8,2,1.5,8000,True,7
9,2,1.5,16000,True,8


In [8]:
def runfolder(row):
    int_part, frac_part = ("%.2f" % row['T']).split('.')
    if row['mcwf']:
        label = 'mcwf_%s' % row['seed']
    else:
        label = 'non_herm'
    rf = 'rf_%dnodes_T%s_%s_%s' % (
        row['n_nodes'], int_part, frac_part, label)
    return join(DATA_ROOT, rf)

In [9]:
def copy_optimized_pulses(from_rf, to_rf):
    for file in glob(join(from_rf, 'pulse*.oct.dat')):
        shutil.copy(file, to_rf)

In [10]:
def task_create_runfolder():
    """Create all necessary runfolders for the runs defined in params_df"""
    slh_models = {}
    runfolders = set()
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        if rf in runfolders:
            raise ValueError("Duplicate runfolder %s, row %d" % (rf, ind))
        runfolders.add(rf)
        n_nodes = int(row['n_nodes'])
        try:
            slh = slh_models[n_nodes]
        except KeyError:
            slh = network_slh(
                n_cavity=2, n_nodes=n_nodes, topology='driven_bs_fb')
            slh_models[n_nodes] = slh
        yield {
            'name': str(rf),
            'actions': [
                (write_dicke_single_model, [slh, ], dict(
                    rf=rf, T=row['T'], theta=0, nt=500, seed=row['seed'],
                    kappa=1.0, E0_cycles=2, mcwf=row['mcwf'], non_herm=True)),
                (copy_optimized_pulses, [SOURCE_RF, rf])
            ],
            'targets': [join(rf, 'config'), join(rf, 'pulse1.oct.dat')],
            'uptodate': [True, ] # up to date if targets exist
        }


In [11]:
N_PROCS = 50

In [12]:
def task_propagate():
    """Run optimization for every runfolder from params_df"""
    for ind, row in params_df.iterrows():
        rf = runfolder(row)
        n_procs = N_PROCS
        if not row['mcwf']:
            n_procs = 1
        yield {
            'name': str(rf),
            'actions': [
                (run_traj_prop, [rf, ], dict(n_trajs=row['n_trajs'], n_procs=n_procs))],
            'file_dep': [join(rf, 'pulse1.oct.dat'),],
            'targets': [join(rf, 'state_final.dat.1'),]}

In [13]:
%doit create_runfolder

.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_non_herm
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_0
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_1
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_2
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_3
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_4
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_5
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_6
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_7
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_8
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_9
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_10
.  create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcw

In [14]:
%doit propagate

-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_non_herm
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_0
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_1
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_2
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_3
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_4
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_5
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_6
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_7
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_8
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_9
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_10
-- create_runfolder:./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcw

In [28]:
def get_err_from_P_target(rf):
    tgrid, P, var_P = np.genfromtxt(join(rf, "P_target.dat"), unpack=True)
    return 1.0-P[-1]

In [15]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_non_herm", 'state_final.dat*')

In [16]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_0", 'state_final.dat*')

In [30]:
get_err_from_P_target("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_0")

In [17]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_1", 'state_final.dat*')

In [31]:
get_err_from_P_target("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_1")

In [18]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_2", 'state_final.dat*')

In [19]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_3", 'state_final.dat*')

In [20]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_4", 'state_final.dat*')

In [21]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_5", 'state_final.dat*')

In [22]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_6", 'state_final.dat*')

In [23]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_7", 'state_final.dat*')

In [24]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_8", 'state_final.dat*')

In [32]:
get_err_from_P_target("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_8")

In [33]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_9", 'state_final.dat*')

In [34]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_10", 'state_final.dat*')

In [35]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_11", 'state_final.dat*')

In [36]:
err_dicke_single("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_12", 'state_final.dat*')

In [37]:
get_err_from_P_target("./data/dicke1_compare_evaluations/rf_2nodes_T1_50_mcwf_12")